In [16]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv


In [17]:
load_dotenv()

True

In [34]:
from langchain.llms import OpenAI

In [36]:
llm = OpenAI(api_key=os.getenv('OPENAI_KEY'), model="gpt-3.5-turbo",temperature=0.5)